In [1]:
# for protein structural modelling
from Bio.SVDSuperimposer import SVDSuperimposer
import numpy as np
import biovec
import glob

# from utils functions
from utils.encoder_decoder import *
from utils.sequence import *
from utils.reward import *
from utils.environment import *

# for deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# for envronment creation
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import MultiDiscrete
from gymnasium.spaces import Discrete
from gymnasium.spaces import Box

#for reading PDB files and processing them
from biopandas.pdb import PandasPdb
import pandas as pd
from utils.sequence import *

# for generating structures through esm instead of modeller
import esm
import biotite.structure as struc
import biotite.structure.io as strucio

# for general utility
import random
import os
import subprocess
import time
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
class PolicyNetwork():
    def __init__(self, n_state, n_action, n_hidden=50,lr=0.001,entropy_weight=0.01):
        self.model = nn.Sequential(nn.Linear(n_state, n_hidden),
                                   nn.ReLU(),
                                   nn.Linear(n_hidden, n_action),
                                   nn.Softmax(dim=-1), )
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        self.entropy_weight = entropy_weight
    def predict(self, s):
        return self.model(torch.Tensor(s))
    def update(self, returns, log_probs,entropies):
        policy_gradient = []
        for log_prob, Gt, entropy in zip(log_probs, returns, entropies):
            policy_gradient.append((-log_prob * Gt) + (self.entropy_weight * entropy))
        loss = torch.stack(policy_gradient).sum()
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
    def get_action(self, s):

        probs = self.predict(s)
        action = torch.multinomial(probs, 1).item()
        log_prob = torch.log(probs[action])
        entropy = -torch.sum(probs * torch.log(probs + 1e-9))  # Calculate entropy
        return action, log_prob, entropy

In [3]:

def reinforce(env, estimator, n_episode, gamma=1.0):
    total_reward_episode = [0] * n_episode

    for episode in range(n_episode):
        log_probs = []
        rewards = []
        entropies = []
        state, info,dummy = env.reset()
        while True:
            action, log_prob, entropy = estimator.get_action(state)
            next_state, reward, terminated, truncated, info = env.step(action)
            total_reward_episode[episode] += reward
            log_probs.append(log_prob)
            rewards.append(reward)
            entropies.append(entropy)
            if terminated or truncated:
                returns = []
                Gt = 0
                pw = 0
                for reward in rewards[::-1]:
                    Gt += gamma ** pw * reward
                    pw += 1
                    returns.append(Gt)
                returns = returns[::-1]
                returns = torch.tensor(returns)
                if returns.std() != 0:
                    returns = (returns - returns.mean()) / (returns.std() + 1e-9)
                estimator.update(returns, log_probs,entropies)
                print('Episode: {}, total reward: {}'.format(episode, total_reward_episode[episode]))
                break
            state = next_state

    return total_reward_episode
                
                    



In [4]:
env = PeptideEvolution(folder_containing_pdb_files='../DrugResistance/folder_for_machine_learning/tp_30_test/',
                       structure_generator='esm_sse',
                       validation=False,
                       reward_cutoff=40,
                       unique_path_to_give_for_file = 'unique_1',
                       maximum_number_of_allowed_mutations_per_episode=15,
                       folder_to_save_validation_files='validation_structures')


In [5]:
n_state = env.observation_space.shape[0]
n_action = env.action_space.n
n_hidden = 128
lr = 0.0007
gamma = 0.95
entropic_factor = 0.001
n_episode = 8000


In [12]:
policy_net = PolicyNetwork(n_state, n_action, n_hidden, lr,entropy_weight=entropic_factor)

In [7]:
total_reward_episode = reinforce(env,estimator=policy_net,n_episode=n_episode,gamma=gamma)

Episode: 0, total reward: 9.96
Episode: 1, total reward: -25.14
Episode: 2, total reward: -25.14
Episode: 3, total reward: -25.14
Episode: 4, total reward: -25.14
Episode: 5, total reward: -25.14
Episode: 6, total reward: -25.14
Episode: 7, total reward: -25.14
Episode: 8, total reward: 9.88
Episode: 9, total reward: -25.14
Episode: 10, total reward: 9.91
Episode: 11, total reward: 9.91
Episode: 12, total reward: -25.14
Episode: 13, total reward: 9.88
Episode: 14, total reward: 9.94
Episode: 15, total reward: 9.86
Episode: 16, total reward: -25.14
Episode: 17, total reward: 9.89
Episode: 18, total reward: 9.94
Episode: 19, total reward: 9.99
Episode: 20, total reward: 9.94
Episode: 21, total reward: -25.14
Episode: 22, total reward: -25.14
Episode: 23, total reward: -25.14
Episode: 24, total reward: -25.14
Episode: 25, total reward: 9.92
Episode: 26, total reward: -25.14
Episode: 27, total reward: 20
Episode: 28, total reward: -25.14
Episode: 29, total reward: -25.14
Episode: 30, total

Episode: 244, total reward: 9.86
Episode: 245, total reward: 9.93
Episode: 246, total reward: -25.14
Episode: 247, total reward: -25.14
Episode: 248, total reward: 9.95
Episode: 249, total reward: 9.91
Episode: 250, total reward: -25.14
Episode: 251, total reward: 9.87
Episode: 252, total reward: 9.91
Episode: 253, total reward: -25.14
Episode: 254, total reward: -25.14
Episode: 255, total reward: 9.92
Episode: 256, total reward: 20
Episode: 257, total reward: 9.86
Episode: 258, total reward: 9.96
Episode: 259, total reward: 9.91
Episode: 260, total reward: -25.14
Episode: 261, total reward: 9.95
Episode: 262, total reward: 9.95
Episode: 263, total reward: -25.14
Episode: 264, total reward: -25.14
Episode: 265, total reward: -25.14
Episode: 266, total reward: -25.14
Episode: 267, total reward: 9.99
Episode: 268, total reward: -25.14
Episode: 269, total reward: 9.93
Episode: 270, total reward: -25.14
Episode: 271, total reward: -25.14
Episode: 272, total reward: 9.87
Episode: 273, total

Episode: 488, total reward: -25.14
Episode: 489, total reward: -25.14
Episode: 490, total reward: 9.9
Episode: 491, total reward: 9.95
Episode: 492, total reward: -25.14
Episode: 493, total reward: 9.93
Episode: 494, total reward: 20
Episode: 495, total reward: -25.14
Episode: 496, total reward: 9.95
Episode: 497, total reward: 9.96
Episode: 498, total reward: 9.94
Episode: 499, total reward: 9.91
Episode: 500, total reward: 9.86
Episode: 501, total reward: 9.95
Episode: 502, total reward: -25.14
Episode: 503, total reward: 9.99
Episode: 504, total reward: 9.95
Episode: 505, total reward: 9.92
Episode: 506, total reward: 20
Episode: 507, total reward: 9.92
Episode: 508, total reward: 9.93
Episode: 509, total reward: 9.88
Episode: 510, total reward: 9.98
Episode: 511, total reward: 9.98
Episode: 512, total reward: 9.94
Episode: 513, total reward: 9.94
Episode: 514, total reward: 9.89
Episode: 515, total reward: 9.87
Episode: 516, total reward: 9.9
Episode: 517, total reward: 9.98
Episod

Episode: 735, total reward: 9.9
Episode: 736, total reward: 20
Episode: 737, total reward: -25.14
Episode: 738, total reward: 9.88
Episode: 739, total reward: 9.98
Episode: 740, total reward: 20
Episode: 741, total reward: 9.86
Episode: 742, total reward: 9.93
Episode: 743, total reward: 9.92
Episode: 744, total reward: 9.95
Episode: 745, total reward: 9.99
Episode: 746, total reward: 9.92
Episode: 747, total reward: -25.14
Episode: 748, total reward: 9.95
Episode: 749, total reward: 9.94
Episode: 750, total reward: 9.93
Episode: 751, total reward: 9.91
Episode: 752, total reward: 20
Episode: 753, total reward: -25.14
Episode: 754, total reward: 9.94
Episode: 755, total reward: 9.98
Episode: 756, total reward: 9.96
Episode: 757, total reward: 9.96
Episode: 758, total reward: 9.95
Episode: 759, total reward: 9.94
Episode: 760, total reward: 9.98
Episode: 761, total reward: 9.97
Episode: 762, total reward: 9.88
Episode: 763, total reward: 9.92
Episode: 764, total reward: 9.86
Episode: 76

Episode: 983, total reward: -25.14
Episode: 984, total reward: 9.96
Episode: 985, total reward: 9.98
Episode: 986, total reward: 9.99
Episode: 987, total reward: 9.97
Episode: 988, total reward: 9.99
Episode: 989, total reward: 9.91
Episode: 990, total reward: 20
Episode: 991, total reward: 9.93
Episode: 992, total reward: 9.91
Episode: 993, total reward: 9.95
Episode: 994, total reward: -25.14
Episode: 995, total reward: -25.14
Episode: 996, total reward: 9.89
Episode: 997, total reward: -25.14
Episode: 998, total reward: -25.14
Episode: 999, total reward: 9.92
Episode: 1000, total reward: 9.9
Episode: 1001, total reward: 9.97
Episode: 1002, total reward: 9.98
Episode: 1003, total reward: 9.93
Episode: 1004, total reward: 9.97
Episode: 1005, total reward: 9.91
Episode: 1006, total reward: 9.94
Episode: 1007, total reward: 9.97
Episode: 1008, total reward: 9.9
Episode: 1009, total reward: 9.93
Episode: 1010, total reward: 9.95
Episode: 1011, total reward: 9.89
Episode: 1012, total rewa

...

Episode: 14843, total reward: 9.95
Episode: 14844, total reward: 9.98
Episode: 14845, total reward: 9.95
Episode: 14846, total reward: 20
Episode: 14847, total reward: 9.99
Episode: 14848, total reward: 9.98
Episode: 14849, total reward: 9.9
Episode: 14850, total reward: 9.99
Episode: 14851, total reward: 9.99
Episode: 14852, total reward: 9.99
Episode: 14853, total reward: 9.9
Episode: 14854, total reward: 9.94
Episode: 14855, total reward: 9.93
Episode: 14856, total reward: 20
Episode: 14857, total reward: 9.98
Episode: 14858, total reward: 9.95
Episode: 14859, total reward: 9.97
Episode: 14860, total reward: 9.98
Episode: 14861, total reward: 20
Episode: 14862, total reward: 9.92
Episode: 14863, total reward: 9.98
Episode: 14864, total reward: 9.98
Episode: 14865, total reward: -25.14
Episode: 14866, total reward: 9.94
Episode: 14867, total reward: -25.14
Episode: 14868, total reward: 9.97
Episode: 14869, total reward: 9.99
Episode: 14870, total reward: 9.97
Episode: 14871, total re

Episode: 15077, total reward: 20
Episode: 15078, total reward: 9.99
Episode: 15079, total reward: 9.88
Episode: 15080, total reward: 9.97
Episode: 15081, total reward: 9.88
Episode: 15082, total reward: 9.98
Episode: 15083, total reward: 9.95
Episode: 15084, total reward: 9.98
Episode: 15085, total reward: 9.97
Episode: 15086, total reward: 9.96
Episode: 15087, total reward: 9.99
Episode: 15088, total reward: 9.92
Episode: 15089, total reward: 20
Episode: 15090, total reward: 9.95
Episode: 15091, total reward: 9.97
Episode: 15092, total reward: 9.94
Episode: 15093, total reward: 9.88
Episode: 15094, total reward: 9.98
Episode: 15095, total reward: 9.97
Episode: 15096, total reward: 9.93
Episode: 15097, total reward: 9.91
Episode: 15098, total reward: 9.98
Episode: 15099, total reward: 9.99
Episode: 15100, total reward: 9.98
Episode: 15101, total reward: 9.97
Episode: 15102, total reward: 9.94
Episode: 15103, total reward: 20
Episode: 15104, total reward: 9.98
Episode: 15105, total rewa

Episode: 15312, total reward: 9.95
Episode: 15313, total reward: 9.9
Episode: 15314, total reward: 9.97
Episode: 15315, total reward: 9.92
Episode: 15316, total reward: 9.92
Episode: 15317, total reward: 9.92
Episode: 15318, total reward: 9.88
Episode: 15319, total reward: 9.93
Episode: 15320, total reward: 9.94
Episode: 15321, total reward: 9.92
Episode: 15322, total reward: 20
Episode: 15323, total reward: 9.97
Episode: 15324, total reward: 9.95
Episode: 15325, total reward: 9.97
Episode: 15326, total reward: 9.97
Episode: 15327, total reward: 9.99
Episode: 15328, total reward: 9.98
Episode: 15329, total reward: 9.95
Episode: 15330, total reward: 9.96
Episode: 15331, total reward: 9.97
Episode: 15332, total reward: 9.94
Episode: 15333, total reward: 9.91
Episode: 15334, total reward: 9.95
Episode: 15335, total reward: 9.95
Episode: 15336, total reward: 9.95
Episode: 15337, total reward: 9.99
Episode: 15338, total reward: 9.98
Episode: 15339, total reward: 9.99
Episode: 15340, total r

Episode: 15547, total reward: 9.9
Episode: 15548, total reward: 9.99
Episode: 15549, total reward: 9.97
Episode: 15550, total reward: 9.97
Episode: 15551, total reward: 9.96
Episode: 15552, total reward: 9.98
Episode: 15553, total reward: 9.93
Episode: 15554, total reward: 9.96
Episode: 15555, total reward: 9.91
Episode: 15556, total reward: 9.93
Episode: 15557, total reward: 9.99
Episode: 15558, total reward: 20
Episode: 15559, total reward: 9.97
Episode: 15560, total reward: 9.97
Episode: 15561, total reward: 9.95
Episode: 15562, total reward: 9.95
Episode: 15563, total reward: 9.96
Episode: 15564, total reward: 9.99
Episode: 15565, total reward: 9.91
Episode: 15566, total reward: 20
Episode: 15567, total reward: 9.99
Episode: 15568, total reward: 9.95
Episode: 15569, total reward: 9.94
Episode: 15570, total reward: 9.91
Episode: 15571, total reward: 9.98
Episode: 15572, total reward: 9.92
Episode: 15573, total reward: 20
Episode: 15574, total reward: 9.91
Episode: 15575, total rewar

Episode: 15782, total reward: -25.14
Episode: 15783, total reward: 9.98
Episode: 15784, total reward: 9.91
Episode: 15785, total reward: 9.98
Episode: 15786, total reward: 20
Episode: 15787, total reward: 9.97
Episode: 15788, total reward: 9.98
Episode: 15789, total reward: 20
Episode: 15790, total reward: 9.98
Episode: 15791, total reward: 9.96
Episode: 15792, total reward: 9.96
Episode: 15793, total reward: 9.99
Episode: 15794, total reward: 9.97
Episode: 15795, total reward: 20
Episode: 15796, total reward: 9.99
Episode: 15797, total reward: 20
Episode: 15798, total reward: 9.94
Episode: 15799, total reward: 9.91
Episode: 15800, total reward: 9.92
Episode: 15801, total reward: 9.98
Episode: 15802, total reward: 9.98
Episode: 15803, total reward: 9.96
Episode: 15804, total reward: 9.96
Episode: 15805, total reward: 9.97
Episode: 15806, total reward: 9.99
Episode: 15807, total reward: 9.99
Episode: 15808, total reward: 9.97
Episode: 15809, total reward: 9.94
Episode: 15810, total rewa

In [8]:
torch.save(policy_net.model.state_dict(),f'saved_models/saved_rl_model_1_lr_{lr}_gamma_{gamma}_ep_{n_episode}_entropic_factor_{entropic_factor}.pth')

In [10]:
np.savetxt(f'saved_models/saved_rl_model_1_lr_{lr}_gamma_{gamma}_ep_{n_episode}_entropic_factor_{entropic_factor}.txt',total_reward_episode)